**Here we are installing contractions library in python which is used to expand contractions in text.
contractions are shortened version of words or phases such as don't (short for do not)**

In [1]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.9 MB/s eta 0:00:00


In [5]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [7]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
import contractions
import re
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [8]:
import pandas as pd
from keras.layers import TextVectorization
import numpy as np
from keras.layers import Input, Embedding, LSTM, GRU, Dense
from keras.models import Sequential
import pickle
import locale
import tensorflow as tf
import os

2024-05-21 05:09:17.849799: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 05:09:17.849901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 05:09:17.976246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [13]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
cpu_count()

4

 **here we are reading our data using pandas library.**

In [15]:
import os

# List the contents of the /kaggle/input directory
input_dir = '/kaggle/input'
for root, dirs, files in os.walk(input_dir):
    for name in files:
        print(os.path.join(root, name))


/kaggle/input/training.1600000.processed.noemoticon.csv


In [16]:
path = "/kaggle/input/sentiment140/training/.1600000/.processed/.noemoticon/.csv"

In [21]:
import os

# Specify the absolute file path
file_path = "/kaggle/input/"
files_and_directories = os.listdir(file_path)

# Print the list
print("Files and directories in", file_path)
for item in files_and_directories:
    print(item)


Files and directories in /kaggle/input/
training.1600000.processed.noemoticon.csv


In [24]:
path = "/kaggle/input/training.1600000.processed.noemoticon.csv"

In [25]:
data = pd.read_csv(path, encoding="latin-1",header=None)

**let's see shape(dimension) of data , here we have 16,00,000 twitter data.**

In [26]:
data.shape

(1600000, 6)

**let's see how are data looks like. here the first five rows of data is displayed.**

In [27]:
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


let's rename our columns

In [28]:
data.rename(columns={0:"target",1:"ids",2:"date",3:"flag",4:"user",5:"text"},
            inplace=True)

In [29]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [30]:
data["text"].iloc[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

we will reamove unnessary columns.we only need target and text.text is our input feature vector and target is our output.

In [31]:
reduced_data = data.drop(labels=data.columns[1:5],axis=1)

In [32]:
reduced_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [33]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [34]:
def normalize_tweet(tweet_text):

  return tweet_text.lower()

In [35]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(normalize_tweet,list(reduced_data["text"])))

In [36]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he can't update his facebook by ...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [37]:
def expand_contractions(tweet_text):

  return contractions.fix(tweet_text)

In [38]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(expand_contractions,list(reduced_data["text"])))

In [39]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he cannot update his facebook by...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it is not behaving at all...
Name: text, dtype: object

In [40]:
regex_pattern = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [41]:
def remove_noisy_tokens(tweet_text):

  return re.sub(pattern=regex_pattern,repl=" ",string=tweet_text)

In [42]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_noisy_tokens,list(reduced_data["text"])))

In [43]:
reduced_data["text"].head()

0      twitpic com  y zl awww that is a bummer you ...
1    is upset that he cannot update his facebook by...
2          managed to save   the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4      it is not behaving at all i am mad why am i ...
Name: text, dtype: object

In [44]:
def remove_remaining_noisy_tokens(tweet_text):

  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',repl=" ",string=tweet_text)

In [45]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_data["text"])))

In [46]:
reduced_data["text"].head()

0     twitpic com   zl awww that is   bummer you sh...
1    is upset that he cannot update his facebook by...
2            managed to save the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4     it is not behaving at all   am mad why am   h...
Name: text, dtype: object

In [47]:
def tokenize_tweet_text(tweet_text):
  
  return word_tokenize(tweet_text)

In [48]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(tokenize_tweet_text,list(reduced_data["text"])))

In [49]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, that, is, bummer, you...
1    [is, upset, that, he, can, not, update, his, f...
2    [managed, to, save, the, rest, go, out, of, bo...
3    [my, whole, body, feels, itchy, and, like, its...
4    [it, is, not, behaving, at, all, am, mad, why,...
Name: text, dtype: object

In [50]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))  

In [51]:
def is_stopword(token):
  
    return not(token in en_stop_words)

In [52]:
def remove_stopwords(tweet_text):

  return list(filter(is_stopword,tweet_text))

In [53]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_stopwords,list(reduced_data["text"])))

In [54]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, got,...
1    [upset, update, facebook, texting, cry, result...
2                        [managed, save, rest, bounds]
3                     [body, feels, itchy, like, fire]
4                                      [behaving, mad]
Name: text, dtype: object

In [55]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wn.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wn.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wn.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wn.ADV)
    else:
        return (treebank_tag[0],wn.NOUN)

In [56]:
def get_pos_tag(tweet_text):

    return list(map(get_wnet_pos_tag,pos_tag(tweet_text)))

In [59]:
reduced_data["text"] = list(map(get_pos_tag,list(reduced_data["text"])))

LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/root/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
reduced_data["text"].head()

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_token(token_pos_tuple):

    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [ ]:
def lemmatize_text(tweet_text):

    if len(tweet_text) > 0:
        return list(map(lemmatize_token,tweet_text))
    else:
        return [""]

In [ ]:
reduced_data["text"] = list(map(lemmatize_text,list(reduced_data["text"])))

In [ ]:
reduced_data["text"].head()

In [ ]:
with open("/content/tokenized_data.pkl","wb") as file_handle:

  pickle.dump(reduced_data,file_handle)

In [ ]:
!mv /content/tokenized_data.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/tokenized_data.pkl /content

In [ ]:
with open("/content/tokenized_data.pkl","rb") as file_handle:

  reduced_data = pickle.load(file_handle)

In [ ]:
reduced_data.head()

In [ ]:
max_tokens = 30000
max_sequence_len = max(list(reduced_data["text"].apply(lambda x: len(x))))

In [ ]:
reduced_rows_idx = np.argwhere(np.array(reduced_data["text"].apply(lambda x: len(x))) >= 10)

In [ ]:
reduced_data = reduced_data.iloc[reduced_rows_idx.reshape(reduced_rows_idx.shape[0],)]

In [ ]:
reduced_data["text"] = list(map(lambda x: " ".join(x), list(reduced_data["text"])))

In [ ]:
reduced_data.head()

In [ ]:
filtered_reduced_data = reduced_data[reduced_data["text"] != ""]

In [ ]:
filtered_reduced_data.shape

In [ ]:
vectorize_layer = TextVectorization(max_tokens=max_tokens,output_sequence_length=max_sequence_len)

In [ ]:
vectorize_layer.adapt(data=np.array(filtered_reduced_data["text"]))

In [ ]:
with open("/content/token_integer_mapping.pkl","wb") as file_handle:

  pickle.dump({"config":vectorize_layer.get_config(),
               "weights":vectorize_layer.get_weights()},file_handle)

In [ ]:
!mv /content/token_integer_mapping.pkl /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/token_integer_mapping.pkl /content

In [ ]:
with open("/content/token_integer_mapping.pkl","rb") as file_handle:

  vectorize_layer_attributes = pickle.load(file_handle)

In [ ]:
loaded_vectorization_layer = TextVectorization.from_config(vectorize_layer_attributes["config"])

In [ ]:
loaded_vectorization_layer.set_weights(vectorize_layer_attributes["weights"])

In [ ]:
def vectorize_tweet(raw_tweet):

  return loaded_vectorization_layer(raw_tweet).numpy()

In [ ]:
vectorized_tweets = list(filtered_reduced_data["text"].apply(vectorize_tweet))

In [ ]:
vectorized_train_tweets = vectorized_tweets[0:200000]
vectorized_cv_tweets = vectorized_tweets[200000:]

In [ ]:
np.savez_compressed("./vectorized_train_tweets.npz",*vectorized_train_tweets)
np.savez_compressed("./vectorized_cv_tweets.npz",*vectorized_cv_tweets)

In [ ]:
!mv /content/vectorized_train_tweets.npz /content/drive/MyDrive
!mv /content/vectorized_cv_tweets.npz /content/drive/MyDrive

In [ ]:
!cp /content/drive/MyDrive/vectorized_train_tweets.npz /content
!cp /content/drive/MyDrive/vectorized_cv_tweets.npz /content

In [ ]:
filtered_reduced_data["target"] = filtered_reduced_data["target"].apply(lambda x: str(x))

In [ ]:
tweet_labels = list(filtered_reduced_data["target"].replace(to_replace=filtered_reduced_data["target"].unique(),
                                        value=list(range(len(filtered_reduced_data["target"].unique())))))

In [ ]:
train_labels = tweet_labels[0:200000]
cv_labels = tweet_labels[200000:]

In [ ]:
train_mb_size = 1000
num_epochs = 10
train_size = 200000

In [ ]:
loaded_vectorized_train_tweets = np.load("./vectorized_train_tweets.npz")

In [ ]:
def train_datagen():

  for _ in range(num_epochs):

    for i in range(train_size//train_mb_size):

      tweets_mb_list = [loaded_vectorized_train_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*train_mb_size,(i+1)*train_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in train_labels[i*train_mb_size:(i+1)*train_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in train_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

In [ ]:
cv_mb_size = 1577
cv_size = 14193

In [ ]:
loaded_vectorized_cv_tweets = np.load("./vectorized_cv_tweets.npz")

In [ ]:
def cv_datagen():

  for _ in range(num_epochs):

    for i in range(cv_size//cv_mb_size):

      tweets_mb_list = [loaded_vectorized_cv_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*cv_mb_size,(i+1)*cv_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in cv_labels[i*cv_mb_size:(i+1)*cv_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [ ]:
for tweets_mb,tweets_labels in cv_datagen():

  print(tweets_mb.shape)
  print(tweets_labels.shape)

  break

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip /content/glove.6B.zip

In [ ]:
max_vocabulary_size = len(loaded_vectorization_layer.get_vocabulary())
embedding_output_dim = 50

In [ ]:
def create_bin_class_rnn():

  rnn_model = Sequential()

  rnn_model.add(Input(shape=(None,),dtype="int64"))
  rnn_model.add(Embedding(input_dim=max_vocabulary_size,output_dim=embedding_output_dim,input_length=max_sequence_len))
  rnn_model.add(LSTM(units=50))
  rnn_model.add(Dense(units=1,activation="sigmoid"))

  return rnn_model

In [ ]:
rnn_model = create_bin_class_rnn()

In [ ]:
rnn_model.summary()

In [ ]:
rnn_model.compile(loss="binary_crossentropy",metrics=["accuracy",
                                                      tf.keras.metrics.Precision(),
                                                      tf.keras.metrics.Recall()])

In [ ]:
training_data_gen = train_datagen()
cv_data_gen = cv_datagen()

In [ ]:
rnn_model.fit(training_data_gen,
              epochs=num_epochs,
              validation_data=cv_data_gen,
              steps_per_epoch=200,validation_steps=9)